# Ibis MapD Backend - Test Numeric

In [1]:
from ibis import literal as L
from ibis.tests.backends import MapD

import ibis
import math
import numpy as np
import pandas as pd

ibis.options.interactive = False

In [2]:
mapd_cli = ibis.mapd.connect(
    host='localhost', user='mapd', password='HyperInteractive',
    port=9091, database='ibis_testing'
)
backend = MapD(None)

In [3]:
def param(ibis_fn, pandas_fn, id, skip=False):
    if skip:
        return None, None, None
    return ibis_fn, pandas_fn, id

In [4]:
alltypes = mapd_cli.database().functional_alltypes
df = alltypes.execute()

In [5]:
tests_1_attr = [
    'year', 'month', 'day',
]
for attr in tests_1_attr:
    try:
        print('{} ... '.format(attr), end='')
        expr = getattr(alltypes.timestamp_col.date(), attr)()
        expected = getattr(df.timestamp_col.dt, attr).astype('int32')

        result = expr.execute()
        expected = backend.default_series_rename(expected)

        backend.assert_series_equal(result, expected)
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
            print('UnsupportedOperationError')

year ... UnsupportedOperationError
month ... UnsupportedOperationError
day ... UnsupportedOperationError


In [6]:
tests_2_attr = [
    'year', 'month', 'day',
    'hour', 'minute', 'second'
]

for attr in tests_2_attr:
    try:
        print('{} ... '.format(attr), end='')
        expr = getattr(alltypes.timestamp_col, attr)()
        expected = getattr(df.timestamp_col.dt, attr).astype('int32')

        result = expr.execute()
        expected = backend.default_series_rename(expected)

        backend.assert_series_equal(result, expected)
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
            print('UnsupportedOperationError')

year ... OK!
month ... OK!
day ... OK!
hour ... OK!
minute ... OK!
second ... OK!


In [7]:
tests_3_unit = [
    'Y', 'M', 'D', 'W',
    'h', 'm', 's', 'ms', 'us', 'ns'
]

for unit in tests_3_unit:
    try:
        print('{} ... '.format(unit), end='')
        expr = alltypes.timestamp_col.truncate(unit)

        dtype = 'datetime64[{}]'.format(unit)
        expected = pd.Series(df.timestamp_col.values.astype(dtype))

        result = expr.execute()
        expected = backend.default_series_rename(expected)

        backend.assert_series_equal(result, expected)
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
        print('UnsupportedOperationError')
    except Exception as e:
        if unit in ['W', 'ns']:
            print('FAILED EXPECTED!')
        else:
            print('FAIL', e)

Y ... OK!
M ... OK!
D ... OK!
W ... FAILED EXPECTED!
h ... OK!
m ... OK!
s ... OK!
ms ... OK!
us ... OK!
ns ... FAILED EXPECTED!


In [8]:
tests_4_unit = ['Y', 'M', 'D', 'W']

for unit in tests_4_unit:
    try:
        print('{} ... '.format(unit), end='')
        expr = alltypes.timestamp_col.date().truncate(unit)

        dtype = 'datetime64[{}]'.format(unit)
        expected = pd.Series(df.timestamp_col.values.astype(dtype))

        result = expr.execute()
        expected = backend.default_series_rename(expected)

        backend.assert_series_equal(result, expected)
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
        print('UnsupportedOperationError')
    except Exception as e:
        if unit in ['W', 'ns']:
            print('FAILED EXPECTED!')
        else:
            print('FAIL', e)

Y ... UnsupportedOperationError
M ... UnsupportedOperationError
D ... UnsupportedOperationError
W ... UnsupportedOperationError


In [9]:
tests_5_unit = [
    'Y', 'Q', 'M', 'W', 'D', 'h', 'm', 's', 'ms', 'us'
]
for unit in tests_5_unit:
    try:
        print('{} ... '.format(unit), end='')
        interval = alltypes.int_col.to_interval(unit=unit)
        expr = alltypes.timestamp_col + interval
        result = mapd_cli.execute(expr)

        def convert_to_offset(x):
            resolution = '{}s'.format(interval.type().resolution)
            return pd.offsets.DateOffset(**{resolution: x})

        offset = df.int_col.apply(convert_to_offset)
        expected = df.timestamp_col + offset
        expected = backend.default_series_rename(expected)

        backend.assert_series_equal(result, expected)
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
        print('UnsupportedOperationError')
    except Exception as e:
        if unit in ['Q']:
            print('FAIL EXPECTED!')
        else:
            print(e)

Y ... 

/home/xmn/miniconda3/envs/ibis/lib/python3.6/site-packages/pandas/core/ops.py:556: PerformanceWarning: Adding/subtracting array of DateOffsets to Series not vectorized
  "Series not vectorized", PerformanceWarning)


OK!
Q ... FAIL EXPECTED!
M ... 

/home/xmn/miniconda3/envs/ibis/lib/python3.6/site-packages/pandas/core/ops.py:556: PerformanceWarning: Adding/subtracting array of DateOffsets to Series not vectorized
  "Series not vectorized", PerformanceWarning)


OK!
W ... OK!
D ... OK!
h ... OK!
m ... OK!
s ... OK!
ms ... UnsupportedOperationError
us ... UnsupportedOperationError


In [10]:
tests_6_unit = ['Y', 'Q', 'M', 'W', 'D']

for unit in tests_6_unit:
    try:
        print('{} ... '.format(unit), end='')
        interval = alltypes.int_col.to_interval(unit=unit)
        array = alltypes.date_string_col.split('/')
        month, day, year = array[0], array[1], array[2]
        date_col = expr = ibis.literal('-').join([
            '20' + year, month, day
        ]).cast('date')
        expr = date_col + interval
        result = mapd_cli.execute(expr)

        def convert_to_offset(x):
            resolution = '{}s'.format(interval.type().resolution)
            return pd.offsets.DateOffset(**{resolution: x})

        offset = df.int_col.apply(convert_to_offset)
        expected = pd.to_datetime(df.date_string_col) + offset
        expected = backend.default_series_rename(expected)

        backend.assert_series_equal(result, expected)
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
        print('UnsupportedOperationError')
    except Exception as e:
        if unit in ['Q']:
            print('FAIL EXPECTED!')
        else:
            print(e)

Y ... UnsupportedOperationError
Q ... UnsupportedOperationError
M ... UnsupportedOperationError
W ... UnsupportedOperationError
D ... UnsupportedOperationError


In [11]:
tests_7_unit = ['h', 'm', 's', 'ms', 'us']
for unit in tests_7_unit:
    try:
        print('{} ... '.format(unit), end='')
        interval = alltypes.int_col.to_interval(unit=unit)
        array = alltypes.date_string_col.split('/')
        month, day, year = array[0], array[1], array[2]
        date_col = ibis.literal('-').join(['20' + year, month, day]).cast('date')
        date_col + interval
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
        print('UnsupportedOperationError')
    except Exception as e:
        if unit in ['Q']:
            print('FAIL EXPECTED!')
        else:
            print(e)

h ... unsupported operand type(s) for +: 'DateColumn' and 'IntervalColumn'
m ... unsupported operand type(s) for +: 'DateColumn' and 'IntervalColumn'
s ... unsupported operand type(s) for +: 'DateColumn' and 'IntervalColumn'
ms ... unsupported operand type(s) for +: 'DateColumn' and 'IntervalColumn'
us ... unsupported operand type(s) for +: 'DateColumn' and 'IntervalColumn'


In [12]:
date_value = pd.Timestamp('2017-12-31')
timestamp_value = pd.Timestamp('2018-01-01 18:18:18')


tests_8 = [
    param(lambda t, be: t.timestamp_col + ibis.interval(days=4),
          lambda t, be: t.timestamp_col + pd.Timedelta(days=4),
          id='timestamp-add-interval'),
    param(lambda t, be: t.timestamp_col - ibis.interval(days=17),
          lambda t, be: t.timestamp_col - pd.Timedelta(days=17),
          id='timestamp-subtract-interval'),
    param(lambda t, be: t.timestamp_col.date() + ibis.interval(days=4),
          lambda t, be: t.timestamp_col.dt.floor('d') + pd.Timedelta(days=4),
          id='date-add-interval'),
    param(lambda t, be: t.timestamp_col.date() - ibis.interval(days=14),
          lambda t, be: t.timestamp_col.dt.floor('d') - pd.Timedelta(days=14),
          id='date-subtract-interval'),
    param(lambda t, be: t.timestamp_col - ibis.timestamp(timestamp_value),
          lambda t, be: pd.Series(
            t.timestamp_col.sub(timestamp_value).values.astype(
                'timedelta64[{}]'.format(be.returned_timestamp_unit))),
          id='timestamp-subtract-timestamp'),
    param(lambda t, be: t.timestamp_col.date() - ibis.date(date_value),
          lambda t, be: t.timestamp_col.dt.floor('d') - date_value,
          id='date-subtract-date'),
]

for expr_fn, expected_fn, test_id in tests_8:
    try:
        print('{} ... '.format(test_id), end='')
        expr = expr_fn(alltypes, backend)
        expected = expected_fn(df, backend)

        result = mapd_cli.execute(expr)
        expected = backend.default_series_rename(expected)

        backend.assert_series_equal(result, expected)
        print('OK!')
    except ibis.common.UnsupportedOperationError as e:
        print('UnsupportedOperationError')
    except Exception as e:
        if unit in ['Q']:
            print('FAIL EXPECTED!')
        else:
            print('FAIL', e)

timestamp-add-interval ... OK!
timestamp-subtract-interval ... OK!
date-add-interval ... UnsupportedOperationError
date-subtract-interval ... UnsupportedOperationError
timestamp-subtract-timestamp ... UnsupportedOperationError
date-subtract-date ... UnsupportedOperationError
